In [74]:
import pickle
import numpy as np
import os
import plotly
import pandas as pd

def getFamilyName(model_name):
    if 'densenet' in model_name.lower():
        model_family='DenseNet'
    elif 'inceptionresnet' in model_name.lower():
        model_family='InceptionResNet'
    elif 'efficientnet' in model_name.lower():
        model_family='EfficientNet'
    elif 'inception' in model_name.lower():
        model_family='Inception'
    elif 'mobilenet' in model_name.lower():
        model_family =  'MobileNet'
    elif 'nasnet' in model_name.lower():
        model_family =  'NASNet'
    elif 'resnet' in model_name.lower():
        model_family =  'ResNet'
    elif 'vgg' in model_name.lower():
        model_family = 'VGG'
    elif 'xception' in  model_name.lower():
        model_family = 'Xception'
    elif 'yolo' in model_name.lower():
        model_family = 'YOLO8'
    else :
        model_family = ''
    return model_family

def approx_USB_power_draw(row):
    if row['backend'] == 'tflite_edgetpu' or row['backend'] == 'NCS2':
        return row['power_draw'] + 25 * (row['running_time']/3600) # assume constant 25 watt
    else:
        return row['power_draw'] 


with open(os.getcwd()+'/result_databases/workstation_classification.pkl', 'rb') as f:
    tmp_data = pickle.load(f)
    tmp_data.drop(tmp_data.index[(tmp_data['architecture'] == 'Workstation') & (tmp_data['batch_size']==32)],inplace=True)

    data = tmp_data
with open(os.getcwd()+'/result_databases/raspi_classification.pkl', 'rb') as f:
    data = data.append( pickle.load(f) )
with open(os.getcwd()+'/result_databases/workstation_segmentation.pkl', 'rb') as f:
    data = data.append( pickle.load(f) )
#with open(os.getcwd()+'/result_databases/ls8nb010_class1.pkl', 'rb') as f:
#    data = data.append( pickle.load(f).assign(architecture='Intel(R) Core(TM) i7-10610U') )
#with open(os.getcwd()+'/result_databases/ls8nb010_segm1.pkl', 'rb') as f:
#    data = data.append( pickle.load(f).assign(architecture='Intel(R) Core(TM) i7-10610U') )
#with open(os.getcwd()+'/result_databases/ls8nb010_segm2.pkl', 'rb') as f:
#    data = data.append( pickle.load(f).assign(architecture='Intel(R) Core(TM) i7-10610U') )
with open(os.getcwd()+'/result_databases/workstation_classification_batchsize_64.pkl', 'rb') as f:
    data = data.append( pickle.load(f) )



#Add Family of Architecture
data['family']= data['model'].apply(lambda a : getFamilyName(a))
data['approx_USB_power_draw'] = data.apply(lambda row: approx_USB_power_draw(row), axis = 1)


easy_dict = {'Core i7-6700':'Workstation' ,'ARM Cortex-A72':'RasPi' ,'Intel(R) Core(TM) i7-10610U':'Laptop','tf_cpu':'','tflite_edgetpu':'EDGE TPU','NCS2':'NCS2','tflite':'TFLITE'}
color_dict = {'Workstation':{'EDGE TPU':'#c45f00','NCS2':'#5494DA','':'#b10000','TFLITE':'#b10000'},'Laptop':{'EDGE TPU':'#ff6f00','NCS2':'#86CEFA','':'#ff0000','TFLITE':'#ff0000'},'RasPi':{'EDGE TPU':'#633a00','NCS2':'#003396','TFLITE':'#300000','':'#300000'}}
fsize_dict = {'DenseNet121': 33.226576, 'DenseNet169': 58.593032, 'DenseNet201': 82.584656, 'EfficientNetB0': 21.856559999999998, 'EfficientNetB1': 32.186136, 'EfficientNetB2': 37.468056, 'EfficientNetB3': 50.136184, 'EfficientNetB4': 78.916208, 'EfficientNetB5': 123.524456, 'EfficientNetB6': 174.53435199999998, 'EfficientNetB7': 268.416968, 'InceptionResNetV2': 225.206864, 'InceptionV3': 108.98181699999999, 'MobileNetV2': 14.552912, 'MobileNetV3Large': 22.7354, 'MobileNetV3Small': 10.804839999999999, 'NASNetMobile': 23.037831999999998, 'QuickNet': 53.162167999999994, 'QuickNetLarge': 93.740032, 'QuickNetSmall': 50.845368, 'RegNetX32GF': 432.05466099999995, 'RegNetX400MF': 22.289129, 'RegNetX8GF': 158.813061, 'ResNet101': 179.68035999999998, 'ResNet152': 242.954816, 'ResNet50': 102.985688, 'ResNext101': 356.173021, 'ResNext50': 100.488385, 'VGG16': 553.491728, 'VGG19': 574.7381439999999, 'Xception': 91.973528,'yolov8s-seg':22544384,'yolov8n-seg':6501171.2,'yolov8m-seg':43725619.2}
data['architecture'] = data['architecture'].map(easy_dict)
data['backend'] = data['backend'].map(easy_dict)

data['fsize'] = data['model'].map(fsize_dict)

# add missing values that are equal across Models
for index, row in data.iterrows():
    current_model = row['model']
    if data.loc[data.model == current_model,'number_of_operations'].isnull:
        data.loc[data.model == current_model,'number_of_operations']=row['number_of_operations'] 
    data.loc[data.model == current_model,'number_of_unmapped_operations']=row['number_of_unmapped_operations'] 
    data.loc[data.model == current_model,'input_shape']=row['input_shape'] 
    data.loc[data.model == current_model,'total_parameters']=row['total_parameters'] 
    data.loc[data.model == current_model,'trainable_parameters']=row['trainable_parameters'] 
    data.loc[data.model == current_model,'non_trainable_parameters']=row['non_trainable_parameters'] 

data["number_of_operations"]=pd.to_numeric(data["number_of_operations"])
data["number_of_unmapped_operations"]=pd.to_numeric(data["number_of_unmapped_operations"])


data["total_parameters"]=pd.to_numeric(data["total_parameters"])

data["trainable_parameters"]=pd.to_numeric(data["trainable_parameters"])
data["non_trainable_parameters"]=pd.to_numeric(data["non_trainable_parameters"])
data["power_draw"] = data["power_draw"]*3600 # convert to wattseconds
data["approx_USB_power_draw"] = data["approx_USB_power_draw"]*3600
#data.drop(data.index[(data['architecture'] == 'Workstation') & (data['batch_size']==32)&(data['log_name'].str.contains("classification"))],inplace=True)


/tmp/ipykernel_1078547/1204965219.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append( pickle.load(f) )
/tmp/ipykernel_1078547/1204965219.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append( pickle.load(f) )
/tmp/ipykernel_1078547/1204965219.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append( pickle.load(f) )


In [75]:
print(data['backend'].unique())
print(data.columns)

['NCS2' '' 'EDGE TPU' nan 'TFLITE']
Index(['log_name', 'task', 'dataset', 'model', 'backend', 'software',
       'architecture', 'number_of_operations', 'number_of_unmapped_operations',
       'input_shape', 'total_parameters', 'trainable_parameters',
       'non_trainable_parameters', 'running_time', 'power_draw',
       'validation_size', 'batch_size', 'accuracy_k1', 'accuracy_k3',
       'accuracy_k5', 'accuracy_k10', 'precision_B', 'recall_B', 'mAP50_B',
       'mAP50_95_B', 'precision_M', 'recall_M', 'mAP50_M', 'mAP50_95_M',
       'family', 'approx_USB_power_draw', 'fsize'],
      dtype='object')


## Divide into Classification and Segmentation, Group

In [76]:
import math
classification_data = data[data['log_name'].str.contains("classification")].reset_index()
segmentation_data = data[data['log_name'].str.contains("segmentation")].reset_index()
classification_data_grouped = classification_data.groupby(['model','backend','architecture']).mean().reset_index()
classification_data_grouped=classification_data_grouped[['model','backend','architecture','accuracy_k1']]
classification_data_grouped.dropna(subset=['accuracy_k1'], inplace=True)

segmentation_data = data[data['log_name'].str.contains("segmentation")].reset_index()
segmentation_data_grouped = segmentation_data.groupby(['model','backend','architecture']).mean().reset_index()
segmentation_data_grouped=segmentation_data_grouped[['model','backend','architecture','precision_B']]
segmentation_data_grouped.dropna(subset=['precision_B'], inplace=True)

   


# Remove Redundant Attributes and Export

In [77]:
export_class_df = classification_data.drop(['index','precision_B', 'recall_B', 'mAP50_B','mAP50_95_B', 'precision_M', 'recall_M', 'mAP50_M', 'mAP50_95_M','family'],axis=1)
export_class_df = export_class_df.groupby(['model','backend','architecture']).mean().reset_index()
export_class_df['task'] = 'infer'
export_class_df['dataset'] = 'imagenet'
export_class_df['environment'] = export_class_df['backend'] + export_class_df['architecture']
export_class_df = export_class_df.drop(export_class_df[export_class_df['model'] == 'VGG16'].index)
export_class_df = export_class_df.drop(export_class_df[export_class_df['architecture'] == 'Laptop'].index)

export_class_df.to_pickle(os.path.join(os.getcwd(),'classification_database/database.pkl')) 
export_seg_df = segmentation_data.drop(['index','accuracy_k1', 'accuracy_k3',
       'accuracy_k5', 'accuracy_k10','family'],axis=1)

export_seg_df = export_seg_df.groupby(['model','backend','architecture']).mean().reset_index()
export_seg_df['task'] = 'infer'
export_seg_df['dataset'] = 'coco'
export_seg_df['environment'] = export_seg_df['backend'] + ' ' + export_seg_df['architecture']

export_seg_df.to_pickle(os.path.join(os.getcwd(),'segmentation_database/database.pkl'))

In [78]:
export_class_df.to_csv('class_experiments.csv')
export_seg_df.to_csv('seg_experiments.csv')

In [79]:
export_seg_df 

,model,backend,architecture,number_of_operations,number_of_unmapped_operations,total_parameters,trainable_parameters,non_trainable_parameters,running_time,power_draw,...,mAP50_95_B,precision_M,recall_M,mAP50_M,mAP50_95_M,approx_USB_power_draw,fsize,task,dataset,environment
0,yolov8m-seg,,Workstation,360.0,210.0,27285968.0,NaN,NaN,0.835094,182820.590998,...,0.495104,0.700502,0.581448,0.630327,0.409402,182820.590998,43725619.2,infer,coco,Workstation
1,yolov8m-seg,EDGE TPU,Workstation,360.0,210.0,27285968.0,NaN,NaN,2.213678,183188.562183,...,0.295751,0.550192,0.515461,0.504972,0.312531,183243.904125,43725619.2,infer,coco,EDGE TPU Workstation
2,yolov8m-seg,NCS2,Workstation,360.0,210.0,27285968.0,NaN,NaN,0.573515,108056.159902,...,0.495158,0.698337,0.583006,0.630519,0.409483,108070.497770,43725619.2,infer,coco,NCS2 Workstation
3,yolov8n-seg,,Workstation,294.0,186.0,3409968.0,NaN,NaN,0.287137,56118.882032,...,0.363096,0.617106,0.455898,0.489357,0.304266,56118.882032,6501171.2,infer,coco,Workstation
4,yolov8n-seg,EDGE TPU,Workstation,294.0,186.0,3409968.0,NaN,NaN,0.395863,42159.105680,...,0.205352,0.461424,0.372768,0.352616,0.208577,42169.002256,6501171.2,infer,coco,EDGE TPU Workstation
5,yolov8n-seg,NCS2,Workstation,294.0,186.0,3409968.0,NaN,NaN,0.148547,25058.943949,...,0.363110,0.617472,0.455924,0.489546,0.304297,25062.657619,6501171.2,infer,coco,NCS2 Workstation
6,yolov8s-seg,,Workstation,293.0,185.0,11821056.0,NaN,NaN,0.464128,96774.302888,...,0.443875,0.685922,0.531394,0.576948,0.369698,96774.302888,22544384.0,infer,coco,Workstation
7,yolov8s-seg,EDGE TPU,Workstation,293.0,185.0,11821056.0,NaN,NaN,1.237709,108882.106471,...,0.255304,0.552786,0.432066,0.438112,0.263960,108913.049191,22544384.0,infer,coco,EDGE TPU Workstation
8,yolov8s-seg,NCS2,Workstation,293.0,185.0,11821056.0,NaN,NaN,0.275398,51609.370977,...,0.443832,0.687456,0.530761,0.576935,0.369717,51616.255936,22544384.0,infer,coco,NCS2 Workstation
